# Example 2: FX futures momentum strategy

## Pre-requisites
### 1. If you have not opened the notebook in Colab, select the button below
<a href="https://githubtocolab.com/SIGTechnologies/sigtech-python/blob/master/examples/notebooks/02_FX_Futures_Momentum.ipynb">
    <img src="https://sigtech.com/wp-content/uploads/2023/08/grey_google_colab.svg"></a>

### 2. Enter your API key
After pasting in your API key, you need to run the cell. In Colab, hover your cursor over an individual code cell and click play to run it.
>**Tip**!\
>After pasting in your API key, you can press `CTRL-F9` (Windows) or `⌘-F9` (Mac) to run the entire notebook at once.

In [ ]:
# Install our Python SDK
%pip install sigtech 

# Import OS and our Python SDK
import sigtech.api as sig
import os

# Define your API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# <YOUR_API_KEY> with the API key you have generated. e.g. os.environ['SIGTECH_API_KEY'] = 'sig_A1B2C3D4E5f6g7h8i9'
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

### 3. Set up your Colab environment

In [ ]:
# Install ta as it is not included in Colab
%pip install ta

# Import any additional Python libraries you require.
import datetime as dtm
import pandas as pd
import ta
import numpy as np

### 4. Create a session
After installing our Python SDK, defining your API key, importing any additional Python libraries or functions you require, and setting any default parameters, initialize your session.

In [ ]:
sig.init()

## Introduction to an FX futures momentum strategy
An FX futures momentum strategy is a trading strategy that aims to capitalize on the persistence of trends in currency futures prices. The basic idea behind the strategy is to go long (buy) on currency futures that have exhibited strong positive returns over a certain period and go short (sell) on those with weak returns.

## Our strategy
- We will define a dictionary of FX assets and create a `RollingFutureStrategy` for each asset.
- For each asset's rolling futures strategy, we will calculate historical percentage changes (returns) over time and use these to generate signals for the strategy. After generating the signals, we will rank the strategies in descending order using the rank method. The top 20% will be longed and the bottom 20% will be shorted.
- Finally, the ranked signals for each contract will be combined into a signal DataFrame and input into the `SignalStrategy` class to create the trend following strategy.

## 1. Create the assets

We define our currency futures assets in a dictionary mapping the contract codes to the contract sector for use in a `for` loop to generate a `RollingFutureStrategy` for each asset.

In [ ]:
assets = {
    "AD": "CURNCY",
    "CD": "CURNCY",
    "SF": "CURNCY",
    "EC": "CURNCY",
    "BP": "CURNCY",
    "JY": "CURNCY",
    "NO": "CURNCY",
    "NV": "CURNCY",
    "SE": "CURNCY",
    "DX": "CURNCY"
}


In [ ]:
strategies = {}  

for contract_code, contract_sector in assets.items():
    strategy = sig.RollingFutureStrategy(
        contract_code=contract_code,
        contract_sector=contract_sector,
        currency='USD',
        start_date='2011-01-01',
        rolling_rule='front', 
        # Roll to the contract month with an expiration date closest to the current date.
        front_offset='-4:-1' 
        # Specifies the number of business days before first delivery notice date (or expiry 
        # date for cash settled futures) to start and finish the roll.
    )
    strategies[contract_code] = strategy  # Store the generated strategies in a dictionary.

## 2. Generate signals
The function below is used to generate trading signals and rank them based on their performance. The top 20% will be assigned a value of 1 and signify when to take a long position, conversely the bottom 20% will be assigned a value of -1 and signify when to take a short position. The remaining signals will be given a value of 0 and signify when to be market-neutral.

In [ ]:
signals ={}

# For each strategy, calculate the percentage change in its historical performance over a 61-day 
# period using the pct_change method applied to the strategy's historical data.
for asset, strategy in strategies.items():
    signals[strategy.name] = strategy.history().pct_change(61)

# Concatenate the signals from different strategies into a single DataFrame called signals_df.
signals_df = pd.concat(signals, join="inner", axis=1).dropna()

# Apply the rank method to signals_df. 
# axis=1 parameter means ranking is done horizontally (by date).
# method='max' means if there are ties (signals with equal value), the highest rank will be assigned. 
# ascending=False parameter ensures that higher signal values receive higher ranks.
ranked_signals_df = signals_df.rank(axis=1, method='max', ascending = False)


final_rank_df = signals_df.copy()
pct = .20

top = round(pct*(len(ranked_signals_df.columns)))
bottom = (len(ranked_signals_df.columns) - top)

# For the top-ranked signals, set their values to 1 in final_rank_df, signalling a long position.
final_rank_df[ranked_signals_df <= top] = 1 

# For the bottom-ranked signals, set their values to -1 in final_rank_df, signalling a short position.
final_rank_df[ranked_signals_df > bottom] = -1

# For the signals falling between the top and bottom ranks, set their values to 0 in final_rank_df, 
# signalling a neutral position.
final_rank_df[(ranked_signals_df >= top) & (ranked_signals_df <= bottom)] = 0

The following `.tail` codeblocks show how the original signal DataFrame is transformed to a DataFrame ranking the various signals and then into a DataFrame which is compatible with `SignalStrategy`. 

 > A `SignalStrategy` requires a `signal_input`. This is a pandas DataFrame where the column headers are the instrument names and the values are the signals for each of the instruments. These signals can be either a number of units *or* a weight.

In [ ]:
signals_df.tail(10)

In [ ]:
ranked_signals_df.tail(10)

In [ ]:
final_rank_df.tail(10)

## 3. Backtest the FX futures momentum strategy

Our signal DataFrame (`final_rank_df`) is used as the input for a `SignalStrategy` which rebalances at the end of each month.

In [ ]:
momentum_strategy = sig.SignalStrategy(
    signal_input=final_rank_df,
    currency = 'USD',
    start_date = final_rank_df.first_valid_index().date(),
    rebalance_frequency = 'EOM',
)

We can now get the historical performance of our strategy using `.history()` and plot these values using `.plot()`.

In [ ]:
momentum_strategy.history().plot(figsize=(20,10))